Course Instructor: Bernd Neumayr, JKU

# UE06: SHACL Rules

Complete the **10 tasks (1 point per task)** in the `5. SHACL Rules` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- The data graph and your solution (the shapes graph including the rules) in executable form
- Print out out the the data graph including derived statements (after execution of the rules). 

# Setup

In [1]:
# Install required packages in the current Jupyter kernel
!pip install -q rdflib 
!pip3 install -q pyshacl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.4.1 requires prettytable<1, but you have prettytable 2.5.0 which is incompatible.


In [2]:
# Imports
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace, Dataset
from rdflib.namespace import FOAF , XSD , RDFS 
from rdflib.plugins.sparql.processor import SPARQLResult
from rdflib.namespace import NamespaceManager

from pyshacl import validate

import pandas as pd

def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = [ { var : res[var].n3(graph.namespace_manager) if (isinstance(res[var],URIRef) and use_prefixes) else res[var] for var in results.vars } for res in results ]     
                                        # construct a list of dictionaries, as intermediate format to construct the pandas DataFrame, use prefixes to abbreviate URIs                
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def validation_report_as_dataframe(validation_report):
  df = sparql_select(results_graph,"""
		SELECT  ?focusNode ?resultPath ?value ?sourceConstraintComponent ?sourceShape ?resultMessage
		WHERE
  		{ ?vr	a sh:ValidationResult ;
						sh:focusNode ?focusNode ;
						sh:sourceConstraintComponent ?sourceConstraintComponent ;
						sh:sourceShape ?sourceShape ;
						sh:resultMessage ?resultMessage .					 
				OPTIONAL { ?vr sh:value ?value . }
				OPTIONAL { ?vr sh:resultPath ?resultPath . }
  		}
  """,use_prefixes=True)
  return df

def shacl_validate(dg,sg):
  return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)  
  

def shacl_validate_with_rules(dg,sg):
	return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=True,
      iterate_rules=True, inplace=True,
      js=False,
      debug=False)


# Task 1
The property :hasDescendant represents the transitive closure of property :hasChild.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a SPARQL rule.

In [3]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

	<Elizabeth>  :hasChild  <Charles> .

  <Charles>  :hasChild  <William> , <Harry> .

  <George>  :hasChild  <Elizabeth> .

  <Harry>  :hasChild  <Archie> .

""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

	<PersonShape> a sh:NodeShape ;
  sh:targetSubjectsOf :hasChild ;
  sh:rule [
      a sh:SPARQLRule ;
      sh:prefixes <Prefixes> ;
      sh:construct \"\"\"
            CONSTRUCT { $this :hasDescendant ?x.}
            WHERE { $this (:hasChild)+ ?x .}
      \"\"\" ;
  ] .
""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

@prefix : <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

:George a rdfs:Resource ;
    :hasChild :Elizabeth ;
    :hasDescendant :Archie,
        :Charles,
        :Elizabeth,
        :Harry,
        :William .

:hasChild a rdf:Property ;
    rdfs:subPropertyOf :hasChild .

rdf:type a rdf:Property ;
    rdfs:subPropertyOf rdf:type .

rdfs:subPropertyOf a rdf:Property ;
    rdfs:subPropertyOf rdfs:subPropertyOf .

:Elizabeth a rdfs:Resource ;
    :hasChild :Charles ;
    :hasDescendant :Archie,
        :Charles,
        :Harry,
        :William .

:Charles a rdfs:Resource ;
    :hasChild :Harry,
        :William ;
    :hasDescendant :Archie,
        :Harry,
        :William .

:Harry a rdfs:Resource ;
    :hasChild :Archie ;
    :hasDescendant :Archie .

:William a rdfs:Resource .

:Archie a rdfs:Resource .




# Task 2
The property :hasDescendant represents the transitive closure of property :hasChild.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a triple rule.


In [4]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

	<Elizabeth>  :hasChild  <Charles> .

  <Charles>  :hasChild  <William> , <Harry> .

  <George>  :hasChild  <Elizabeth> .

  <Harry>  :hasChild  <Archie> .

""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

<PersonShape> a sh:NodeShape ;
  sh:targetSubjectsOf :hasChild ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject sh:this ;
      sh:predicate :hasDescendant ;
      sh:object [sh:path :hasChild ]
  ], [
      a sh:TripleRule ;
      sh:subject sh:this ;
      sh:predicate :hasDescendant ;
      sh:object [sh:path ( :hasDescendant :hasDescendant ) ]
  ].
""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

@prefix : <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

:George a rdfs:Resource ;
    :hasChild :Elizabeth ;
    :hasDescendant :Archie,
        :Charles,
        :Elizabeth,
        :Harry,
        :William .

:hasChild a rdf:Property ;
    rdfs:subPropertyOf :hasChild .

rdf:type a rdf:Property ;
    rdfs:subPropertyOf rdf:type .

rdfs:subPropertyOf a rdf:Property ;
    rdfs:subPropertyOf rdfs:subPropertyOf .

:Elizabeth a rdfs:Resource ;
    :hasChild :Charles ;
    :hasDescendant :Archie,
        :Charles,
        :Harry,
        :William .

:Charles a rdfs:Resource ;
    :hasChild :Harry,
        :William ;
    :hasDescendant :Archie,
        :Harry,
        :William .

:Harry a rdfs:Resource ;
    :hasChild :Archie ;
    :hasDescendant :Archie .

:William a rdfs:Resource .

:Archie a rdfs:Resource .




# Task 3
The tax rate of a product category is propagated to products.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a triple rule.

In [5]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<HP4>   rdf:type          <Product> ;
        :price            12 ;
        :productCategory  <Book> .

<Porsche911>  rdf:type    <Product> ;
        :price            121000 ;
        :productCategory  <Car> .

<VolvoV50>  rdf:type      <Product> ;
        :price            27000 ;
        :productCategory  <Car> .

<Car>   :taxRate  0.2 .

<Book>  :taxRate  0.1 .



""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

<TaxRateShape> a sh:NodeShape ;
  sh:targetClass <Product> ;
  sh:rule [
    a sh:TripleRule ;
    sh:subject sh:this ;
    sh:predicate :taxRate ;
    sh:object [sh:path ( :productCategory :taxRate )]
  ] .

""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

@prefix : <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:HP4 a :Product,
        rdfs:Resource ;
    :price 12 ;
    :productCategory :Book ;
    :taxRate 0.1 .

:Porsche911 a :Product,
        rdfs:Resource ;
    :price 121000 ;
    :productCategory :Car ;
    :taxRate 0.2 .

:VolvoV50 a :Product,
        rdfs:Resource ;
    :price 27000 ;
    :productCategory :Car ;
    :taxRate 0.2 .

:Book a rdfs:Resource ;
    :taxRate 0.1 .

:price a rdf:Property ;
    rdfs:subPropertyOf :price .

:productCategory a rdf:Property ;
    rdfs:subPropertyOf :productCategory .

:taxRate a rdf:Property ;
    rdfs:subPropertyOf :taxRate .

rdf:type a rdf:Property ;
    rdfs:subPropertyOf rdf:type .

rdfs:subPropertyOf a rdf:Property ;
    rdfs:subPropertyOf rdfs:subPropertyOf .

12 a rdfs:Resource .

27000 a rdfs:Resource .

121000 a rdfs:Resource .

:Car a r

# Task 4
Properties :hasMother and :hasFather are derived from properties :hasChild and classes Man and Woman.
Additional requirements (not checked by the tool, you have to check them yourself):
Derive :hasFather statements using a triple rule defined as part of a node shape that has Man as target class.
Derive :hasMother statements using a triple rule defined as part of a node shape that has Woman as target class.

In [6]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Diana>  rdf:type  <Woman> ;
        :hasChild  <William> , <Harry> .

<William>  rdf:type  <Man> .

<Harry>  rdf:type  <Man> ;
        :hasChild  <Archie> .

<Man>   rdfs:subClassOf  <Person> .

<Charles>  rdf:type  <Man> ;
        :hasChild  <William> , <Harry> .

<Archie>  rdf:type  <Person> .

<Elizabeth>  rdf:type  <Woman> ;
        :hasChild  <Charles> .

<Woman>  rdfs:subClassOf  <Person> .



""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

<MotherShape> a sh:NodeShape ;
  sh:targetClass <Woman> ;
  sh:rule [
    a sh:TripleRule ;
    sh:subject [sh:path :hasChild ] ;
    sh:predicate :hasMother ;
    sh:object sh:this ; 
  ] .

<FatherShape> a sh:NodeShape ;
  sh:targetClass <Man> ;
  sh:rule [
    a sh:TripleRule ;
    sh:subject [sh:path :hasChild ] ;
    sh:predicate :hasFather ;
    sh:object sh:this ; 
  ] .
""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

@prefix : <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

:Archie a :Person,
        rdfs:Resource ;
    :hasFather :Harry .

:Elizabeth a :Person,
        :Woman,
        rdfs:Resource ;
    :hasChild :Charles .

:hasChild a rdf:Property ;
    rdfs:subPropertyOf :hasChild .

rdf:type a rdf:Property ;
    rdfs:subPropertyOf rdf:type .

rdfs:subClassOf a rdf:Property ;
    rdfs:subPropertyOf rdfs:subClassOf .

rdfs:subPropertyOf a rdf:Property ;
    rdfs:subPropertyOf rdfs:subPropertyOf .

:Diana a :Person,
        :Woman,
        rdfs:Resource ;
    :hasChild :Harry,
        :William .

:William a :Man,
        :Person,
        rdfs:Resource ;
    :hasFather :Charles ;
    :hasMother :Diana .

:Woman a rdfs:Resource ;
    rdfs:subClassOf :Person .

:Charles a :Man,
        :Person,
        rdfs:Resource ;
    :hasChild :Harry,
        :William ;
    :hasMother :Elizabeth .

:Harry a :Man,
    

# Task 5
Properties :hasMother and :hasFather are derived from properties :hasChild and classes Man and Woman.
Additional requirements (not checked by the tool, you have to check them yourself):
Derive :hasFather statements as well as :hasMother statements using triple rules defined as part of a node shape that has Person as target class.

In [14]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Diana>  rdf:type  <Woman> ;
        :hasChild  <William> , <Harry> .

<William>  rdf:type  <Man> .

<Harry>  rdf:type  <Man> ;
        :hasChild  <Archie> .

<Man>   rdfs:subClassOf  <Person> .

<Charles>  rdf:type  <Man> ;
        :hasChild  <William> , <Harry> .

<Archie>  rdf:type  <Person> .

<Elizabeth>  rdf:type  <Woman> ;
        :hasChild  <Charles> .

<Woman>  rdfs:subClassOf  <Person> .



""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

<PersonShape> a sh:NodeShape ;
  sh:targetClass <Person> ;
  sh:rule [
      a sh:TripleRule ;
      sh:subject [ sh:path :hasChild ] ;
      sh:predicate :hasFather ;
      sh:object sh:this ;
      sh:condition [ sh:class <Man> ]
  ], [
      a sh:TripleRule ;
      sh:subject [ sh:path :hasChild ] ;
      sh:predicate :hasMother ;
      sh:object sh:this ;
      sh:condition [ sh:class <Woman> ]
  ].
""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

RuleLoadError: ignored

# Task 6
The average weight of a species should be aggregated from the individual animals.
(This can be solved using a SPARQL rule)

In [8]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Collie>  rdfs:subClassOf  <Dog> .

<Garfield>  rdf:type  <Cat> ;
        :weight   12 .

<Bello>  rdf:type  <Dog> ;
        :weight   27 .

<Cat>   rdf:type  <Species> .

<Lassie>  rdf:type  <Collie> ;
        :weight   31 .

<Tom>   rdf:type  <Cat> ;
        :weight   6 .

<Dog>   rdf:type  <Species> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

<AvgWeightShape> a sh:NodeShape ;
  sh:targetClass <Species> ;
  sh:rule [
    a sh:SPARQLRule ;
    sh:prefixes <Prefixes> ;
    sh:construct \"\"\"
      CONSTRUCT { $this :avgWeight ?avg .}
      WHERE {
         SELECT (avg(?x) as ?avg)
         WHERE {
           ?y rdf:type $this ; :weight ?x .
         }
      } 
    \"\"\"
  ] .
""")

# In SemAI.jar leicht anders weil Lassie nicht als Hunde categorisiert wird (?y rdf:type/rdfs:subClassOf* $this ; :weight ?x .)
conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

@prefix : <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Bello a :Dog,
        rdfs:Resource ;
    :weight 27 .

:Garfield a :Cat,
        rdfs:Resource ;
    :weight 12 .

:Lassie a :Collie,
        :Dog,
        rdfs:Resource ;
    :weight 31 .

:Tom a :Cat,
        rdfs:Resource ;
    :weight 6 .

:Collie a rdfs:Resource ;
    rdfs:subClassOf :Dog .

:weight a rdf:Property ;
    rdfs:subPropertyOf :weight .

rdf:type a rdf:Property ;
    rdfs:subPropertyOf rdf:type .

rdfs:subClassOf a rdf:Property ;
    rdfs:subPropertyOf rdfs:subClassOf .

rdfs:subPropertyOf a rdf:Property ;
    rdfs:subPropertyOf rdfs:subPropertyOf .

6 a rdfs:Resource .

12 a rdfs:Resource .

27 a rdfs:Resource .

31 a rdfs:Resource .

:Cat a :Species,
        rdfs:Resource ;
    :avgWeight 9.0 .

:Species a rdfs:Resource .

:Dog a :Species,
        rdfs:Resource ;
  

# Task 7
The area of a rectangle is the product of its length and its width.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a SPARQL rule


In [9]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :length   1.72 ;
        :width    .9 .

<Rect2>  rdf:type  <Rectangle> ;
        :length   5.5 ;
        :width    4 .

<Rect1>  rdf:type  <Rectangle> ;
        :length   10 ;
        :width    6 .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

<AreaShape> a sh:NodeShape ;
  sh:targetClass <Rectangle> ;
  sh:rule [
    a sh:SPARQLRule ;
    sh:prefixes <Prefixes> ;
    sh:construct \"\"\"
      CONSTRUCT { $this :area ?area . }
      WHERE { 
        $this :length ?x; :width ?y . 
        BIND(?x * ?y AS ?area) 
      }
    \"\"\"
  ].
""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

@prefix : <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Peter a :Person,
        rdfs:Resource ;
    :length 1.72 ;
    :width 0.9 .

:Rect1 a :Rectangle,
        rdfs:Resource ;
    :area 60.0 ;
    :length 10 ;
    :width 6 .

:Rect2 a :Rectangle,
        rdfs:Resource ;
    :area 22.0 ;
    :length 5.5 ;
    :width 4 .

:Person a rdfs:Resource .

:length a rdf:Property ;
    rdfs:subPropertyOf :length .

:width a rdf:Property ;
    rdfs:subPropertyOf :width .

rdf:type a rdf:Property ;
    rdfs:subPropertyOf rdf:type .

rdfs:subPropertyOf a rdf:Property ;
    rdfs:subPropertyOf rdfs:subPropertyOf .

0.9 a rdfs:Resource .

1.72 a rdfs:Resource .

4 a rdfs:Resource .

5.5 a rdfs:Resource .

6 a rdfs:Resource .

10 a rdfs:Resource .

:Rectangle a rdfs:Resource .




# Task 8
The area of a rectangle is the product of its length and its width.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a SHACL function :multiply together with a triple rule.

In [10]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :length   1.72 ;
        :width    .9 .

<Rect2>  rdf:type  <Rectangle> ;
        :length   5.5 ;
        :width    4 .

<Rect1>  rdf:type  <Rectangle> ;
        :length   10 ;
        :width    6 .

""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""

PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

<AreaShape> a sh:NodeShape ;
  sh:targetClass <Rectangle> ;
  sh:rule [
    a sh:TripleRule ;
    sh:subject sh:this ;
    sh:predicate :area ;
    sh:object [ :multiply( [ sh:path :width] [ sh:path :length] ) ] ;
  ] .

:multiply
	a sh:SPARQLFunction ;
	rdfs:comment "Multiplies its two arguments $op1 and $op2." ;
	sh:parameter [
		sh:path :op1 ;
		sh:datatype xsd:integer ;
		sh:description "The first operand" ;
	] ;
	sh:parameter [
		sh:path :op2 ;
		sh:datatype xsd:integer ;
		sh:description "The second operand" ;
	] ;
	sh:returnType xsd:integer ;
	sh:select \"\"\"
		SELECT ($op1 * $op2 AS ?result)
		WHERE {
		}
		\"\"\" .
""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

@prefix : <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Peter a :Person,
        rdfs:Resource ;
    :length 1.72 ;
    :width 0.9 .

:Rect1 a :Rectangle,
        rdfs:Resource ;
    :area 60.0 ;
    :length 10 ;
    :width 6 .

:Rect2 a :Rectangle,
        rdfs:Resource ;
    :area 22.0 ;
    :length 5.5 ;
    :width 4 .

:Person a rdfs:Resource .

:length a rdf:Property ;
    rdfs:subPropertyOf :length .

:width a rdf:Property ;
    rdfs:subPropertyOf :width .

rdf:type a rdf:Property ;
    rdfs:subPropertyOf rdf:type .

rdfs:subPropertyOf a rdf:Property ;
    rdfs:subPropertyOf rdfs:subPropertyOf .

0.9 a rdfs:Resource .

1.72 a rdfs:Resource .

4 a rdfs:Resource .

5.5 a rdfs:Resource .

6 a rdfs:Resource .

10 a rdfs:Resource .

:Rectangle a rdfs:Resource .




# Task 9
The body mass index (:bmi) of a person is its weight divided by its height squared.
The bmi should be rounded to 2 decimal places.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using a SPARQL rule.

In [11]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Bello>  rdf:type  <Dog> ;
        :height   0.77 ;
        :weight   27 .

<Peter>  rdf:type  <Person> ;
        :height   1.78 ;
        :weight   86 .

<Jane>  rdf:type  <Person> ;
        :height   1.72 ;
        :weight   72 .


""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""

PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

<BMIShape> a sh:NodeShape ;
  sh:targetClass <Person> ;
  sh:rule [
    a sh:SPARQLRule ;
    sh:prefixes <Prefixes> ;
    sh:construct \"\"\"
      CONSTRUCT { $this :bmi ?bmi . }
      WHERE { 
       $this :weight ?x; :height ?y . 
       BIND(ROUND(?x*100 / (?y * ?y))/100 AS ?bmi)
      }
    \"\"\"
  ].
""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

@prefix : <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Bello a :Dog,
        rdfs:Resource ;
    :height 0.77 ;
    :weight 27 .

:Jane a :Person,
        rdfs:Resource ;
    :bmi 24.34 ;
    :height 1.72 ;
    :weight 72 .

:Peter a :Person,
        rdfs:Resource ;
    :bmi 27.14 ;
    :height 1.78 ;
    :weight 86 .

:Dog a rdfs:Resource .

:height a rdf:Property ;
    rdfs:subPropertyOf :height .

:weight a rdf:Property ;
    rdfs:subPropertyOf :weight .

rdf:type a rdf:Property ;
    rdfs:subPropertyOf rdf:type .

rdfs:subPropertyOf a rdf:Property ;
    rdfs:subPropertyOf rdfs:subPropertyOf .

0.77 a rdfs:Resource .

1.72 a rdfs:Resource .

1.78 a rdfs:Resource .

27 a rdfs:Resource .

72 a rdfs:Resource .

86 a rdfs:Resource .

:Person a rdfs:Resource .




# Task 10
The body mass index (:bmi) of a person is its weight divided by its height squared.
The bmi should be rounded to 2 decimal places.
Additional requirements (not checked by the tool, you have to check them yourself):
Solve this using SHACL functions (one for multiply and one for division rounded to two decimal places) and a triple rule.

In [12]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Bello>  rdf:type  <Dog> ;
        :height   0.77 ;
        :weight   27 .

<Peter>  rdf:type  <Person> ;
        :height   1.78 ;
        :weight   86 .

<Jane>  rdf:type  <Person> ;
        :height   1.72 ;
        :weight   72 .


""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""

PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   <http://example.org/>

<BMIShape> a sh:NodeShape ;
  sh:targetClass <Person> ;
  sh:rule [
    a sh:TripleRule ;
    sh:subject sh:this ;
    sh:predicate :bmi ;
    sh:object [ 
       :round (
         [:divide (
           [sh:path :weight]
           [:multiply ( [sh:path :height] [sh:path :height] )]
         )]
       )
    ] ;
  ].

:multiply
	a sh:SPARQLFunction ;
	rdfs:comment "Multiplies its two arguments $op1 and $op2." ;
	sh:parameter [
		sh:path :op1 ;
		sh:datatype xsd:integer ;
		sh:description "The first operand" ;
	] ;
	sh:parameter [
		sh:path :op2 ;
		sh:datatype xsd:integer ;
		sh:description "The second operand" ;
	] ;
	sh:returnType xsd:integer ;
	sh:select \"\"\"
		SELECT ($op1 * $op2 AS ?result)
		WHERE {
		}
		\"\"\" .

:divide
	a sh:SPARQLFunction ;
	rdfs:comment "Divides its two arguments $op1 and $op2." ;
	sh:parameter [
		sh:path :op1 ;
		sh:datatype xsd:integer ;
		sh:description "The first operand" ;
	] ;
	sh:parameter [
		sh:path :op2 ;
		sh:datatype xsd:integer ;
		sh:description "The second operand" ;
	] ;
	sh:returnType xsd:integer ;
	sh:select \"\"\"
		SELECT ($op1 / $op2 AS ?result)
		WHERE {
		}
		\"\"\" .

:round
	a sh:SPARQLFunction ;
	rdfs:comment "Rounds value" ;
	sh:parameter [
		sh:path :val ;
		sh:datatype xsd:float ;
		sh:description "value to round" ;
	] ;
	sh:returnType xsd:integer ;
	sh:select \"\"\"
		SELECT (ROUND($val*100)/100 AS ?result)
		WHERE {
		}
		\"\"\" .
""")


conforms, results_graph, results_text = shacl_validate_with_rules(dg,sg)
print(dg.serialize(format='turtle'))

@prefix : <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Bello a :Dog,
        rdfs:Resource ;
    :height 0.77 ;
    :weight 27 .

:Jane a :Person,
        rdfs:Resource ;
    :bmi 24.34 ;
    :height 1.72 ;
    :weight 72 .

:Peter a :Person,
        rdfs:Resource ;
    :bmi 27.14 ;
    :height 1.78 ;
    :weight 86 .

:Dog a rdfs:Resource .

:height a rdf:Property ;
    rdfs:subPropertyOf :height .

:weight a rdf:Property ;
    rdfs:subPropertyOf :weight .

rdf:type a rdf:Property ;
    rdfs:subPropertyOf rdf:type .

rdfs:subPropertyOf a rdf:Property ;
    rdfs:subPropertyOf rdfs:subPropertyOf .

0.77 a rdfs:Resource .

1.72 a rdfs:Resource .

1.78 a rdfs:Resource .

27 a rdfs:Resource .

72 a rdfs:Resource .

86 a rdfs:Resource .

:Person a rdfs:Resource .


